In [ ]:
import numpy as np
import pandas as pd
import json
import scipy
import ast
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from ast import literal_eval

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install networkx


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import mlxtend
print(mlxtend.__version__)

0.23.2


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data.csv', sep=',(?=\S)', engine='python')

df['friends'] = [json.loads(x) for x in df['friends']]
df['id'] = [int(str(x).replace('"', '')) for x in df['id']]
df['screenName'] = [x.replace('"', '') for x in df['screenName']]

df = df[df['friendsCount'] < 300]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence '\S'
<>:1: DeprecationWarning: invalid escape sequence '\S'
<ipython-input-11-4166a5e6f880>:1: DeprecationWarning: invalid escape sequence '\S'
  df = pd.read_csv('/content/drive/MyDrive/data.csv', sep=',(?=\S)', engine='python')


In [ ]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,screenName,tags,avatar,followersCount,friendsCount,lang,lastSeen,tweetId,friends
0,1969527638,LlngoMakeEmCum_,"[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/534286217...",319,112,"""en""",1472271687519,"""769310701580083200""","[1969574754, 1969295556, 1969284056, 196961221..."
1,51878493,_notmichelle,"[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/761977602...",275,115,"""en""",1472270622663,"""769309490038439936""","[60789485, 2420931980, 2899776756, 127410795, ..."
2,1393409100,jesseayye,"[ ""#narcos"" ]","""http://pbs.twimg.com/profile_images/713282938...",120,107,"""en""",1472804144409,"""771622623583186944""","[86868062, 19697415, 2998836604, 456295047, 74..."
4,710130422907207680,sarahdorat_16,"[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/767180520...",128,218,"""en""",1472271397356,"""769309769144283137""","[1571896093, 768938323612008448, 2548665930, 3..."
5,3649469655,wanderIustregui,"[ ""#veranomtv2016"" ]","""http://pbs.twimg.com/profile_images/762400806...",479,131,"""en""",1472736978337,"""771340902308671488""","[2401096388, 707864762, 4096348512, 7638558599..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13164 entries, 0 to 39996
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              13164 non-null  int64 
 1   screenName      13164 non-null  object
 2   tags            13164 non-null  object
 3   avatar          13164 non-null  object
 4   followersCount  13164 non-null  int64 
 5   friendsCount    13164 non-null  int64 
 6   lang            13164 non-null  object
 7   lastSeen        13164 non-null  int64 
 8   tweetId         13164 non-null  object
 9   friends         13164 non-null  object
dtypes: int64(4), object(6)
memory usage: 1.1+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# utiliser uqe 10000 lignes
df_temp = df.head(10000)
df_temp['tags'] = df_temp['tags'].apply(lambda x: literal_eval(x) if isinstance(x, str) else [])

te = TransactionEncoder()
te_ary = te.fit(df_temp['tags']).transform(df_temp['tags'])
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df_trans, min_support=0.01, use_colnames=True)
num_itemsets = 10
limited_itemsets = frequent_itemsets.nlargest(num_itemsets, 'support')
print("Itemsets sélectionnés :\n", limited_itemsets)
rules = association_rules(limited_itemsets, metric="lift", min_threshold=0.5,num_itemsets=3)

print("\nRègles d'association :\n", rules)

Itemsets sélectionnés :
    support               itemsets
2   0.7531      (#nationaldogday)
3   0.1551  (#respecttylerjoseph)
0   0.0133         (#gloryoutnow)
1   0.0101              (#narcos)

Règles d'association :
 Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, representativity, leverage, conviction, zhangs_metric, jaccard, certainty, kulczynski]
Index: []


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-13-9c966e6e5b32>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tags'] = df_temp['tags'].apply(lambda x: literal_eval(x) if isinstance(x, str) else [])


Faible taux de covriance entre les tags donc impossibiliter de crée des règles

In [ ]:
print(df['friends'].head())
print(df['friends'].isnull().sum())

0    [1969574754, 1969295556, 1969284056, 196961221...
1    [60789485, 2420931980, 2899776756, 127410795, ...
2    [86868062, 19697415, 2998836604, 456295047, 74...
4    [1571896093, 768938323612008448, 2548665930, 3...
5    [2401096388, 707864762, 4096348512, 7638558599...
Name: friends, dtype: object
0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df_temp = df.head(5000)
def convert_to_list(friends_str):
    try:
        return ast.literal_eval(friends_str) if isinstance(friends_str, str) else friends_str
    except (ValueError, SyntaxError):
        return []

df['friends'] = df['friends'].apply(lambda x: convert_to_list(x))

all_friends = [friend for sublist in df_temp['friends'] for friend in sublist]
friend_counts = Counter(all_friends)
frequent_friends = {friend for friend, count in friend_counts.items() if count >= 2}

df_temp['filtered_friends'] = df_temp['friends'].apply(lambda x: [friend for friend in x if friend in frequent_friends])
te = TransactionEncoder()
te_ary = te.fit_transform(df_temp['filtered_friends'])
df_trans = pd.DataFrame(te_ary, columns=te.columns_).astype('uint8')
frequent_itemsets = apriori(df_trans, min_support=0.05, use_colnames=True)
num_itemsets = 10
limited_itemsets = frequent_itemsets.nlargest(num_itemsets, 'support')
print("Itemsets sélectionnés :\n", limited_itemsets)

rules = association_rules(limited_itemsets, metric="lift", min_threshold=0.5, num_itemsets=10)
rules = rules.head(20)
print("\nRègles d'association :\n", rules)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-22-d758d1fdd1ce>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['filtered_friends'] = df_temp['friends'].apply(lambda x: [friend for friend in x if friend in frequent_friends])
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might

Itemsets sélectionnés :
      support                         itemsets
25    0.1428                      (221412285)
14    0.1354                       (16712746)
51    0.1344                       (59325073)
80    0.1328            (221412285, 16712746)
97    0.1250            (221412285, 59325073)
32    0.1228                      (266336410)
85    0.1212             (16712746, 59325073)
146   0.1206  (221412285, 16712746, 59325073)
9     0.1194                       (15846407)
17    0.1112                       (17919972)

Règles d'association :
               antecedents            consequents  antecedent support  \
0             (221412285)             (16712746)              0.1428   
1              (16712746)            (221412285)              0.1354   
2             (221412285)             (59325073)              0.1428   
3              (59325073)            (221412285)              0.1344   
4              (16712746)             (59325073)              0.1354   
5           

# Analyse des Résultats : Itemsets Fréquents et Règles d'Association

## 1. **Itemsets Sélectionnés**

Les itemsets fréquents montrent des groupes d'amis (identifiants) apparaissant fréquemment ensemble dans les listes d'amis. Voici quelques observations :

### Itemsets Simples (Un seul identifiant)
- **`221412285`** : Support = **14,28 %**  
  Cet utilisateur apparaît dans environ **14,28 %** des transactions.
- **`16712746`** : Support = **13,54 %**
- **`59325073`** : Support = **13,44 %**

### Itemsets Composés (Paires ou triplets)
- **`(221412285, 16712746)`** : Support = **13,28 %**  
  Ces deux amis apparaissent ensemble dans **13,28 %** des transactions.
- **`(221412285, 59325073)`** : Support = **12,50 %**
- **`(221412285, 16712746, 59325073)`** : Support = **12,06 %**  
  Indique que ces trois amis sont souvent connectés dans un même groupe.

---

## 2. **Règles d'Association**

### a) **Règle : `221412285 → 16712746`**
- **Confiance** : **92,99 %**  
  Lorsque l'utilisateur `221412285` est présent, `16712746` l'est aussi dans **92,99 %** des cas.
- **Lift** : **6,87**  
  La présence de `221412285` augmente la probabilité de voir `16712746` par un facteur de **6,87** par rapport à une association aléatoire.
- **Conviction** : **12,34**  
  Montre une forte fiabilité de la règle.

### b) **Règle : `16712746 → 59325073`**
- **Confiance** : **89,51 %**  
  Si `16712746` est dans une transaction, `59325073` y sera aussi dans **89,51 %** des cas.
- **Lift** : **6,66**  
  Montre une forte dépendance entre ces deux amis.

### c) **Règle : `(16712746, 59325073) → 221412285`**
- **Confiance** : **99,50 %**  
  Si les deux amis `16712746` et `59325073` sont présents, il y a presque une certitude (**99,5 %**) que `221412285` soit aussi dans la transaction.
- **Lift** : **6,97**  
  Indique une forte corrélation entre ces amis.
- **Conviction** : **173,15**  
  Cette règle est extrêmement fiable.

---

## 3. **Interprétation Globale**
- **Réseau social dense** : Les identifiants sélectionnés appartiennent probablement à un groupe d'amis très connectés, formant des sous-communautés.
- **Relations fortement associées** : La **confiance élevée** et les **lifts supérieurs à 6** montrent que ces amis apparaissent presque toujours ensemble.
- **Règles robustes** : Les métriques comme **conviction** et **zhang's metric** confirment la fiabilité des relations mises en évidence.

---

## 4. **Améliorations Possibles**
- **Segmenter par groupe** : Identifier si ces amis appartiennent à une communauté spécifique pourrait fournir des insights plus profonds.
- **Baisser le support minimal** : Cela pourrait permettre de découvrir des sous-groupes d'amis moins fréquents mais potentiellement intéressants.

